# DashStyle Fusion
#### Apply the style of one ArcGIS Dashboard to other ArcGIS Dashboards
created by Niklas Köhn, Esri Deutschland, 2024

In [2]:
from arcgis.gis import GIS
import getpass

# user input username + password
username = input("Username: ")
password = getpass.getpass(prompt='Password: ', stream=None) # hide password

# login to ArcGIS Online or ArcGIS Enterprise
gis = GIS("https://www.arcgis.com", username, password)
print("Successfully logged in as: " + gis.properties.user.username)

Username: esride-nik
Password: ········
Successfully logged in as: esride-nik


In [ ]:
# user select to search org or own account
print("1. Search in the organization")
print("2. Search in my account")
print("3. Exit")
choice = input("Enter your choice: ")

# search for dashboards in the org or own account
if choice == "1":
    search = input("Enter search term: ")
    items = gis.content.search(query=search, item_type="Dashboard", max_items=100)
    print("Found " + str(len(items)) + " dashboards")
    for item in items:
        print(item.title + " (" + item.id + ")")
elif choice == "2":
    search = input("Enter search term: ")
    items = gis.content.search(query=search, item_type="Dashboard", max_items=100, outside_org=True)
    print("Found " + str(len(items)) + " dashboards")
    for item in items:
        print(item.title + " (" + item.id + ")")
else:
    exit()

1. Search in the organization
2. Search in my account
3. Exit
